<table align="left">
  <td>
    <a href="https://tinyurl.com/2khrsfbc" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# Natural Language Processing with TensorFlow

El objetivo principal del [procesamiento de lenguaje natural](https://tinyurl.com/y68tj6v6) (NLP) es analizar, comprender y derivar información valiosa de texto. Las aplicaciones de NLP pueden clasificarse en dos categorías principales:

- Texto (Como el que se encuentra en un email, un tweet, un documento, etc.)
- Audio (Como el que se encuentra en una grabación de voz, una canción, etc.)

Por ejemplo, 

- Si tuviéramos un conjunto de datos de texto que contuviera reseñas de películas, podríamos usar NLP para identificar el sentimiento detrás de cada reseña (por ejemplo, positivo o negativo). 
- También podríamos usar NLP para identificar el tema de cada reseña (por ejemplo, romance, acción, etc.). Si quisieramos construir un identificador de spam, podríamos usar NLP para identificar palabras clave en un correo electrónico que podrían indicar que es spam.
- Generación de texto: Podríamos usar NLP para generar texto, como un resumen de un artículo, un título de un artículo, una respuesta a un correo electrónico, etc.

Cuando se trabaja con texto o audio en NLP los datos tienen un orden secuencial por lo tanto el tratamiento que reciben los datos es diferente al de los datos de imágenes o tabulares.

**Aplicaciones del Modelado de Datos Secuenciales**

Al hablar de redes neuronales densas se tenía una información de entrada y se esperaba una salida sin importar el tiempo ni el orden en el que llega la información a la red. En el modelado de datos secuenciales, el orden de la información es importante:

1. En el primer ejemplo se tienen varias entradas y se obtiene una salida. 
2. En el segundo ejemplo se tiene una entrada y se quiere obtener una secuencia que represente información asociada a la acción entrante. 
3. En el ultimo ejemplo se tiene una secuencia de entrada y se espera obtener una secuencia de salida.

<a href="https://ibb.co/m0LKSmf"><img src="https://i.ibb.co/jzcYHdn/nlp-problems.png" alt="nlp-problems" border="0"></a>

Uno de los factores a tener en cuenta al trabajar con datos secuenciales, es que las redes neuronales densas no son capaces de capturar la información de secuencia. Por lo tanto, se requiere una red neuronal que sea capaz de capturar la información de secuencia. Para ello se utilizan las **redes neuronales recurrentes**.

Las **redes neuronales recurrentes (RNN**): son redes neuronales que tienen una memoria interna que les permite capturar la información de secuencia. La arquitecura base de las RNN es la siguiente:

<a href="https://ibb.co/dg8ycMy"><img src="https://i.ibb.co/9tQjvgj/arq-rnn.png" alt="arq-rnn" border="0"></a>

<a href="https://ibb.co/TrkZFZS"><img src="https://i.ibb.co/yVXDbD1/imagen-2023-02-13-105448726.png" alt="imagen-2023-02-13-105448726" border="0"></a>

>**En este notebook, se trabará sobre lo siguiente**:

- Descargar y explorar un conjunto de datos de texto
- Visualizar datos de texto
- Convertir texto en números usando tokenización
- Convertir texto tokenizado en vectores usando codificación embebida
- Modelar un dataset de texto
- Predecir el sentimiento de una oración
    - Iniciar con una referencia (TF-IDF)
    - Construir diferentes modelos de texto de deep learning
        - Dense, LSTM, GRU, Conv1D, Transfer Learning
- Comparar el desempeño de cada modelo
- Combinar los en un ensamble
- Guardar y cargar un modelo entrenado
- Encontrar las peores predicciones.

<a href="https://ibb.co/yYkdqg3"><img src="https://i.ibb.co/7RpjKr9/nlp-experiments.png" alt="nlp-experiments" border="0"></a>

In [1]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 2060 with Max-Q Design (UUID: GPU-9970422a-f4b7-7ab0-4f13-419e26e877d3)


## 1. Descargar y explorar un dataset de texto

Se utilizará el dataset [Real or Not?](https://www.kaggle.com/c/nlp-getting-started/data) que contiene Tweets acerca de desastres naturales. El objetivo es predecir si un tweet es acerca de un desastre real o no.

- Un Tweet **Real** es acerca de un desastre real, por ejemplo:
> Jetstar and Virgin forced to cancel Bali flights again because of ash from Mount Raung volcano

- Un Tweet **No Real** no es acerca de un desastre real pueden ser sobre cualquier cosa, por ejemplo:
> 'Education is the most powerful weapon which you can use to change the world.' Nelson #Mandela #quote

Al descomprimir el dataset se obtendran tres archivos:

- `sample_submission.csv` - un archivo de muestra de la estructura de envío de predicciones en las competencias de Kaggle.
- `train.csv` - el conjunto de datos de entrenamiento que contiene los tweets reales y no reales de desastres etiquetados.
- `test.csv` - el conjunto de datos de prueba que contiene los tweets reales y no reales que se deben predecir.


### 1.1 Visualizar un dataset de texto

Una vez que se cuenta con un dataset para trabajar, es una buena idea visualizarlo para ver si hay algo que se pueda entender de él. El dataset se encuentra en archivos `.csv`, una forma sencilla de hacerlos legibles es usando la función `pd.read_csv()` de pandas.

In [2]:
# Turn .csv files into pandas DataFrame's
import pandas as pd
train_df = pd.read_csv("datasets/nlp_real_not_dataset/train.csv")
test_df = pd.read_csv("datasets/nlp_real_not_dataset/test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


los datos descargados seguramente se encuentran mezclados. Pero para asegurarnos es una buena idea volverlos a mezclar.

In [3]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) # shuffle with random_state=42 for reproducibility
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


Los datos de entrenamiento contienen la columna `target` que indica si un tweet es real o no real. Esta columna será la etiqueta que se usará para entrenar el modelo. El dataset de prueba no contiene esta columna, ya que es lo que se debe predecir.

La estructura que se quiere implementar es la siguiente:

> Inputs (text column) -> Machine Learning Algorithm -> Outputs (target column)

<img src="https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/08-text-classification-inputs-and-outputs.png" alt="nlp" border="0" width = 800>

In [4]:
# The test data doesn't have a target (that's what we'd try to predict)
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

Dado que se cuenta con dos valores objetivo (0 y 1), se trata de un problema de clasificación binaria. El dataset se encuentra desbalanceado, con un 60% de tweets de clase 0 y un 40% de tweets de clase 1.

donde: 

- 1 = Un tweet es acerca de un desastre real
- 0 = Un tweet no es acerca de un desastre real

El dataset de entrenamiento contiene 7613 tweets, el dataset de prueba contiene 3263 tweets.

In [6]:
# How many samples total?
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [7]:
# Visualización de algunos de los ejemplos de entrenamiento
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
    _,text, target = row
    print(f"Target: {target}", "(real disaster)"
          if target>0 else "(not real disaster)"  
    )
    print(f"Text:\n{text}\n")
    print("---\n")


Target: 0 (not real disaster)
Text:
I'm not the mom friend but I still see my friends as my little babies that I have to care for or else they'll electrocute themselves

---

Target: 0 (not real disaster)
Text:
@OKgooner hahaha great song. 'Spent 15 years getting loaded. 15 years till his liver exploded. Now what's Bob going to do NOW that he...'

---

Target: 1 (real disaster)
Text:
Three Israeli soldiers wounded in West Bank terrorist attack via /r/worldnews http://t.co/su4ZVWADj7

---

Target: 0 (not real disaster)
Text:
#hot  Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/wvTPuRYx63 #prebreak #best

---

Target: 1 (real disaster)
Text:
How is it one careless match can start a forest fire but it takes a whole box to start a campfire?

---



### 1.2 Separar el dataset en conjuntos de entrenamiento y validación

Debido a que el dataset de prueba no contiene la columna `target`, se debe separar el dataset de entrenamiento en dos conjuntos: uno para entrenar el modelo y otro para validar el modelo. Para ello se utilizará la función [`train_test_split()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) de sklearn.

In [8]:
# split data in train and validation
from sklearn.model_selection import train_test_split
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) # random state for reproducibility


In [9]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [10]:
# verificar los primeros 10 ejemplos
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

### 1.3 Convertir texto en números

Para poder alimentar texto a un modelo de deep learning, es necesario convertirlo en números. Hay varias formas de hacer esto, las más comunes son:

- **Tokenización**: Mapeado directo de palabras, caracteres o símbolos a números enteros. Hay tres niveles de tokenización:
    - **Palabra**: convertir cada palabra en una secuencia en un número entero. Con la sentencia "i love tensorflow", la palabra "i" se convertiría en el número 1, "love" en el número 2 y "tensorflow" en el número 3.
    - **Caracter**: convertir cada caracter en una secuencia en un número entero. Como convertir el abecedarios A-Z en números del 1 al 26.
    - **Subpalabra**: convertir cada subpalabra en una secuencia en un número entero. Involucra dividir las palabras en subpalabras, por ejemplo, "tensorflow" se dividiría en "ten", "sor", "flo", "w".


- **Codificación embebida**: Es una representación de lenguaje natural que puede ser aprendida. La representación se da en forma de un vector de características de longitud fija, donde cada característica representa un aspecto del lenguaje. Por ejemplo, la palabra "dance" puede ser presentada por un vector de 5 dimensiones [-0.8547, 0.4559, -0.3332, 0.9877, 0.1112]. Es importante resaltar que el tamaño del vector es ajustable. Existen dos formas de usar la codificación embebida:
    - **Codificación embebida preentrenada**: Se utiliza una codificación embebida que ya ha sido entrenada en un conjunto de datos grande. Por ejemplo, la codificación embebida preentrenada de [GloVe](https://nlp.stanford.edu/projects/glove/).
    - **Codificación embebida entrenada**: Se entrena una codificación embebida desde cero en un conjunto de datos específico. Por ejemplo, se entrena una codificación embebida en un conjunto de datos de tweets.

<a href="https://ibb.co/0sJqwkD"><img src="https://i.ibb.co/Z2TdZ3L/tokens.png" alt="tokens" border="0"></a>

>:key: **Nota**: Qué nivel de tokenización o codificación embebida se debe utilizar? 

Depende del problema que se está tratando de resolver. Es importante experimentar con diferentes niveles de tokenización y codificación embebida para ver cuál funciona mejor para tu problema.

Algunas herramientas de codificación embebida preentrenadas:

- [GloVe](https://nlp.stanford.edu/projects/glove/)
- [Word2Vec](https://jalammar.github.io/illustrated-word2vec/)

>:toolbox: **Vectorización de texto (tokenización)**

Para vectorizar el texto, se utilizará la clase [`TextVectorization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization) de TensorFlow. Esta clase convierte una secuencia de caracteres en una secuencia de enteros, donde cada entero representa un token en un diccionario.

La capa `TextVectorization` toma los siguientes parametros: 

- `max_tokens` - el número máximo de tokens a tener en el diccionario. Si se establece en `None`, el diccionario tendrá tantos tokens como palabras únicas en el texto de entrenamiento.
- `standardize` - la función de normalización a aplicar a los datos de entrada. Por defecto, la función de normalización convierte el texto a minúsculas y elimina la puntuación.
- `split` - la función de separación a aplicar a los datos de entrada. Por defecto, "whitespace" separa el texto en palabras.
- `ngrams` - Cuantas palabras se deben combinar en un solo token. Por ejemplo, si `ngrams` es 2, la frase "I love TensorFlow" se convertiría en "I love", "love TensorFlow".
- `output_mode` - el modo de salida de la capa. Por defecto, "int" devuelve una secuencia de enteros (cada entero representa un token en el diccionario). `int` (mapeado entero), `binary` (one-hot encoding), `count` o `tf-idf`.
- `output_sequence_length` - Tamaño de la secuencia de salida tokenizada. Por ejemplo, si `output_sequence_length` es 10, la secuencia de salida tendrá 10 tokens, si la secuencia de entrada tiene menos de 10 tokens, se rellenará con ceros. Si la secuencia de entrada tiene más de 10 tokens, se truncará a los primeros 10 tokens.
- `pad_to_max_tokens` - Si es `True`, la secuencia de salida se rellenará con ceros hasta que tenga el tamaño de `max_tokens`.

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
# Use the default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

La inicialización anterior de la capa `TextVectorization` utiliza los parametros por defecto. A continuación se creará una nueva capa `TextVectorization` con los parametros que se adaptan mejor al problema.

En particular, se configurará los parametros `max_tokens` y `output_sequence_length`. 

- `max_tokens` - se establece en 10000, ya que el diccionario tendrá tantos tokens como palabras únicas en el texto de entrenamiento.
- `output_sequence_length` - se utilizará el numero promedio de tokens por tweet en el dataset de entrenamiento. El numero promedio de tokens por tweet es 15.5, por lo que se establecerá `output_sequence_length` en 15.

In [12]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [13]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [14]:
# Ajustar el vectorizador de texto a los datos
text_vectorizer.adapt(train_sentences)

In [15]:
# Crear una sentencia de ejemplo y tokenizarla
sample_sentence = "there is a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [16]:
# Escoger una muestra aleatoria del dataset de entrenamiento y tokenizarla
random_sentence = random.choice(train_sentences)
print(f"original text:\n {random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

original text:
 I'm feeling so attacked https://t.co/CvkQiGr1AZ        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 32, 707,  28, 323,   1,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [17]:
# obtener las palabras unicas en el vocabulario
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


>:toolbox: **Codificación embebida**

Hasta el momento se ha convertido el texto en números enteros. Sin embargo, los números enteros no son una representación muy útil para un modelo de deep learning. Por lo tanto, se utilizará la codificación embebida para convertir los números enteros en vectores densos, que pueden ser aprendidos por el modelo.

La ventaja de la codificación embebida es que puede ser aprendida durante el entrenamiento. Por lo tanto, la codificación embebida puede capturar información específica del conjunto de datos de entrenamiento. Por ejemplo, si el conjunto de datos de entrenamiento contiene palabras como "amor", "feliz" y "triste", la codificación embebida puede aprender que "amor" y "feliz" están relacionados y "triste" no está relacionado con "amor" o "feliz".

Para codificar el texto, se utilizará la clase [`tf.keras.layers.Embedding`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) de TensorFlow. Esta clase convierte una secuencia de enteros en una secuencia de vectores densos, donde cada entero representa un token en un diccionario.

Los parametros de la clase `Embedding` son:

- `input_dim` - el tamaño del diccionario. Por ejemplo, si el diccionario tiene 10,000 tokens, `input_dim` es 10,000. (El tamaño del diccionario se establece en el parametro `max_tokens` de la capa `TextVectorization`).
- `output_dim` - el tamaño del vector de salida. Por ejemplo, si se establece en 128, cada token se convertirá en un vector de 128 dimensiones.
- `embeddings_initializer` - la inicialización de los vectores de codificación embebida. Por defecto, los vectores de codificación embebida se inicializan con números aleatorios pequeños. Sin embargo, se puede utilizar una codificación embebida preentrenada, como [GloVe](https://nlp.stanford.edu/projects/glove/).
- `input_length` - el tamaño de la secuencia de entrada. Por ejemplo, si `input_length` es 15, la entrada debe ser una secuencia de 15 enteros.

In [18]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

embedding

:key: **Nota**: Se observa que la embedding es una capa de TensorFlow, por lo que se puede utilizar como una capa de entrada en un modelo de Keras.

Ejemplo de algunas sentencias de texto y su codificación embebida:

In [19]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
#hot  Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/J2aQs5loxu #prebreak #best      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00988939, -0.02746936,  0.00140302, ...,  0.03693564,
         -0.02811122, -0.04898233],
        [ 0.00195915, -0.03295889, -0.02535405, ..., -0.04278285,
         -0.00316085,  0.04614298],
        [ 0.04808098, -0.01105697, -0.01183013, ...,  0.03402085,
         -0.00430602,  0.04447634],
        ...,
        [-0.04318868,  0.03441192, -0.0104339 , ...,  0.00982716,
          0.03474809,  0.02626885],
        [ 0.03977952, -0.03782602, -0.03646283, ...,  0.00236253,
          0.03332629,  0.02803668],
        [ 0.02895654,  0.01618583,  0.03750667, ...,  0.0460008 ,
         -0.01629362,  0.00331082]]], dtype=float32)>

In [24]:
# verificar un unico token en codificación embebida
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.00988939, -0.02746936,  0.00140302,  0.00284123, -0.01472485,
        -0.01839679,  0.0118969 ,  0.04235381, -0.00026729, -0.00642087,
         0.02772132,  0.02748709, -0.01417947, -0.03560337,  0.00357419,
        -0.01770699,  0.02152342,  0.03922397, -0.01609625,  0.02858197,
        -0.00755884,  0.01439104,  0.03739473, -0.01915296, -0.01072513,
         0.01270156, -0.01597232, -0.03835863, -0.03827304, -0.00607399,
         0.0488129 ,  0.02241942,  0.01502916,  0.03161443,  0.04319861,
        -0.04185198, -0.00232239, -0.02205428,  0.0469125 , -0.03193682,
        -0.03613007,  0.01355774, -0.00302942, -0.00800582, -0.00873001,
        -0.00984627,  0.0298288 ,  0.00029044,  0.00571251,  0.00692202,
        -0.02142215,  0.03313327, -0.02721083,  0.03188301, -0.01273811,
        -0.03887856, -0.03365548,  0.01385478, -0.03734227, -0.00772613,
         0.04217112,  0.00022364,  0.00648459, -0.0071331 ,  0.04496172,
  

## 2 - Modelo de deep learning para clasificación de texto

- TF-IDF: Term Frequency - Inverse Document Frequency (Frecuencia de término - Frecuencia inversa del documento). Es una medida estadística que se utiliza para evaluar la importancia de una palabra en un documento en un conjunto de documentos.

<a href="https://ibb.co/yYkdqg3"><img src="https://i.ibb.co/7RpjKr9/nlp-experiments.png" alt="nlp-experiments" border="0"></a>


### Model 0: Naive Bayes (baseline)

Como en todos los experiments en machine learning, es importante crear un modelo de baseline para comparar con los modelos futuros.

Para crear un modelo de baseline, se utilizará el algoritmo de Naive Bayes, que es un algoritmo de clasificación de texto muy popular. Se creará un pipeline utilizando scikit-learn utilizando el metodo TF-IDF (Term Frequency - Inverse Document Frequency) para convertir el texto en números y despues de eso modelarlos utilizando el algoritmo Multinomial Naive Bayes. La selección se realizó siguiendo el mapa de algoritmos de machine learning de scikit-learn.

<img src="https://scikit-learn.org/stable/_static/ml_map.png" alt="ml-map" border="0" width=700>